# Operating Room Utilization Rate Calculation for 2017

In this notebook, we calculate the daily operating room utilization rate for each theatre using the cleaned 2017 dataset from Assuta Ramat HaHayal. Our aim is to quantify how effectively scheduled and unscheduled procedure hours fill the total available block time, providing a key performance indicator for capacity planning and efficiency analysis.

**1. Imports and Data Loading**  
- We import essential libraries (`pandas`, `numpy`, `matplotlib.pyplot`, `seaborn`, and relevant statistical functions).  
- We load the pre-cleaned Excel file (`2017_data_Assuta_nomissings_for_stat_final_1105.xlsx`) into a pandas DataFrame.

**2. Initial Inspection and Copy**  
- We review the first rows, column names, and data types to confirm correct parsing.  
- We create a working copy of the DataFrame (`df_copy`) to ensure the original remains intact.

**3. Data Cleaning and Timestamp Construction**  
- We remove “No Show” records that should not contribute to utilization metrics.  
- We convert separate date and time columns (planned start/end, incision, closure) into full `datetime` objects and merge them into unified timestamp fields.

**4. Feature Engineering**  
- **Block Duration (SH)**: We compute each block's total available hours (`SH_r_hours`) from planned start to planned end.  
- **Scheduled Procedure Hours (SP)**: We calculate the limited scheduled service hours per case and aggregate by room and date (`total_SP_per_day_room`).  
- **Unscheduled Procedure Hours (SU)**: We compute limited unscheduled hours per case and aggregate by room and date (`SU_p_limited_hours`).

**5. Utilization Rate Calculation**  
- We define the daily utilization rate as:  
  ```python
  daily_utilization_rate = (total_SP_per_day_room + SU_p_limited_hours) / SH_r_hours * 100

We assign NaN for any days or rooms with zero or missing service hours to avoid division errors.

**6. Merging and Summary Statistics**
We merge the calculated daily_utilization_rate back onto the main DataFrame by block date and operating room number.

We generate summary statistics and distribution charts for SH_r_hours and daily_utilization_rate to inspect room efficiency across the year.

By following these steps, we enrich the 2017 dataset with a robust utilization metric that will support deeper analysis of operating room performance, identify under- or over-utilized days, and guide future capacity-planning decisions.

Calculating the target variable - utilization percentage:

In [ ]:
#Importing the libraries:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
import warnings
import itertools
from pandas.api.types import is_numeric_dtype
from scipy.stats import shapiro

In [ ]:
file_path = r"/content/2017_data_Assuta_nomissings_for_stat_final_1105.xlsx"
df = pd.read_excel(file_path)
print(df.head())

   patient_id  Site Code Main Surgeon Code  Activity Code  Activity Type Code  \
0           1         20       -999, 30817          81.51                  15   
1           2         20       -999, 18731          65.21                  22   
2           7         20       -999, 18796          65.21                  22   
3           9         20       -999, 31448          65.21                  22   
4          26         20       -999, 23054          65.21                  22   

   Planned SU Time (Large/Medium/Small)  Pre-Surgery Patient File (External)  \
0                                  20.0                           23457403.0   
1                                  20.0                           23457802.0   
2                                   0.0                           23457800.0   
3                                  20.0                           23457631.0   
4                                  20.0                           23462326.0   

  Pre-Surgery Admission Date  He

In [ ]:
print(df.columns)

Index(['patient_id', 'Site Code', 'Main Surgeon Code', 'Activity Code',
       'Activity Type Code', 'Planned SU Time (Large/Medium/Small)',
       'Pre-Surgery Patient File (External)', 'Pre-Surgery Admission Date',
       'Height', 'Weight', 'Patient Age (on Surgery Day)',
       'Background Diseases/Diagnoses', 'Planned Surgery Date',
       'Planned Surgery Time', 'Surgery Admission Date',
       'Administrative Admission Time', 'Planned Operating Room Number',
       'Pre-Surgery Hospitalization Admission Date',
       'Pre-Surgery Hospitalization Admission Time',
       'Pre-Surgical Admission Time Before Surgery', 'Surgical Team Codes',
       'Anesthesiologist Code', 'Anesthesia Code', 'Type of Anesthesia',
       'Cancellation Reason on Surgery Day', 'Actual Operating Room Number',
       'Actual Surgery Room Entry Date', 'Actual Surgery Room Entry Time',
       'Incision Time', 'Closure Time', 'End of Surgery Date (Exit from OR)',
       'End of Surgery Time (Exit from OR)', 

In [ ]:
print(df.dtypes)

patient_id                                       int64
Site Code                                        int64
Main Surgeon Code                               object
Activity Code                                  float64
Activity Type Code                               int64
                                                ...   
Recovery Room Entry Time_Minutes                 int64
Recovery Room Exit Time_Minutes                  int64
Post-Surgery Discharge Time_Minutes              int64
Planned Start Time for Doctor Block_Minutes      int64
Planned End Time for Doctor Block_Minutes        int64
Length: 63, dtype: object


Preparing and checking the time and date columns - converting to an appropriate data type

In [ ]:
df_copy=df.copy()

In [ ]:
print(f"Number of rows: {df_copy.shape[0]}")
print(f"Number of columns: {df_copy.shape[1]}")

Number of rows: 14216
Number of columns: 63


In [ ]:
print(df_copy.head(10))

   patient_id  Site Code Main Surgeon Code  Activity Code  Activity Type Code  \
0           1         20       -999, 30817          81.51                  15   
1           2         20       -999, 18731          65.21                  22   
2           7         20       -999, 18796          65.21                  22   
3           9         20       -999, 31448          65.21                  22   
4          26         20       -999, 23054          65.21                  22   
5          38         20              2982          26.33                  19   
6          39         20              2982          26.33                  19   
7          40         20              2982          26.33                  19   
8          41         20              2982          26.33                  19   
9          42         20              2982          26.33                  19   

   Planned SU Time (Large/Medium/Small)  Pre-Surgery Patient File (External)  \
0                           

Handling duplicates and missing values of type NALL:

In [ ]:
# Overall duplicate count (all rows):
num_duplicates = df_copy.duplicated().sum()
print(f"The number of completely repeating lines: {num_duplicates}")

The number of completely repeating lines: 0


We tried to merge the duplicate rows more effectively based on the following parameters: age, weight, height, surgery entry date, and surgeon code. We attempted to prevent data loss by combining the different values from the duplicate rows. However, the code did not work as expected and distorted the time-related data, resulting in incorrect utilization calculations.

Planned End Time for Doctor Block:

In [ ]:
# Print the unique values in a column:
print(df_copy['Planned End Time for Doctor Block'].unique())

['12:44:00' '20:55:00' '12:30:00' '15:21:00' '18:55:00' '10:06:00'
 '08:33:00' '13:12:00' '11:39:00' '09:20:00' '12:17:00' '12:55:00'
 '10:08:00' '15:33:00' '14:33:00' '09:00:00' '14:56:00' '13:04:00'
 '11:09:00' '11:55:00' '11:51:00' '11:54:00' '11:46:00' '11:59:00'
 '11:27:00' '10:55:00' '12:25:00' '11:56:00' '12:00:00' '10:54:00'
 '11:04:00' '11:10:00' '11:40:00' '11:52:00' '12:09:00' '11:35:00'
 '12:47:00' '10:42:00' '12:15:00' '11:19:00' '11:45:00' '10:46:00'
 '12:37:00' '10:37:00' '12:21:00' '10:45:00' '12:49:00' '13:21:00'
 '10:50:00' '09:54:00' '11:03:00' '13:35:00' '09:14:00' '11:00:00'
 '12:54:00' '10:34:00' '16:10:00' '12:38:00' '15:00:00' '14:44:00'
 '10:30:00' '09:49:00' '09:29:00' '12:45:00' '16:00:00' '10:24:00'
 '08:44:00' '12:02:00' '10:20:00' '09:35:00' '13:10:00' '14:00:00'
 '09:37:00' '14:02:00' '13:07:00' '13:40:00' '16:46:00' '14:27:00'
 '13:55:00' '14:30:00' '13:45:00' '11:20:00' '11:30:00' '14:15:00'
 '14:57:00' '14:37:00' '14:50:00' '14:55:00' '12:27:00' '09:44

In [ ]:
print(df_copy['Planned End Time for Doctor Block'].dtype)
print(df_copy['Planned End Time for Doctor Block'].head(10))

object
0    12:44:00
1    20:55:00
2    12:30:00
3    15:21:00
4    18:55:00
5    10:06:00
6    08:33:00
7    13:12:00
8    08:33:00
9    10:06:00
Name: Planned End Time for Doctor Block, dtype: object


In [ ]:
# Convert to datetime format and then save only the time:
df_copy['Planned End Time for Doctor Block'] = pd.to_datetime(
    df_copy['Planned End Time for Doctor Block'],
    format='%H:%M:%S',
    errors='coerce'  # Converts invalid values to NaT
).dt.time

In [ ]:
# Checking the column type and values
print(df_copy['Planned End Time for Doctor Block'].dtype)  # Should return object but with type time
print(df_copy['Planned End Time for Doctor Block'].head(10))
print("Missing values:", df_copy['Planned End Time for Doctor Block'].isna().sum())

object
0    12:44:00
1    20:55:00
2    12:30:00
3    15:21:00
4    18:55:00
5    10:06:00
6    08:33:00
7    13:12:00
8    08:33:00
9    10:06:00
Name: Planned End Time for Doctor Block, dtype: object
Missing values: 0


In [ ]:
df_copy['Planned End Time for Doctor Block'] = pd.to_timedelta(
    df_copy['Planned End Time for Doctor Block'].astype(str),
    errors='coerce')

In [ ]:
print(df_copy['Planned End Time for Doctor Block'].dtype)

timedelta64[ns]


Closure Time:

In [ ]:
print(df_copy['Closure Time'].unique())

['12:55:00' '22:11:00' '12:24:00' ... '01:04:00' '00:30:00' '00:42:00']


In [ ]:
print(df_copy['Closure Time'].dtype)
print(df_copy['Closure Time'].head(10))

object
0    12:55:00
1    22:11:00
2    12:24:00
3    15:22:00
4    17:10:00
5    10:29:00
6    09:00:00
7    12:06:00
8    09:15:00
9    08:33:00
Name: Closure Time, dtype: object


In [ ]:
df_copy['Closure Time'] = pd.to_datetime(
    df_copy['Closure Time'],
    format='%H:%M:%S',
    errors='coerce'
).dt.time

In [ ]:
print(df_copy['Closure Time'].dtype)
print(df_copy['Closure Time'].head(10))
print("Missing values:", df_copy['Closure Time'].isna().sum())

object
0    12:55:00
1    22:11:00
2    12:24:00
3    15:22:00
4    17:10:00
5    10:29:00
6    09:00:00
7    12:06:00
8    09:15:00
9    08:33:00
Name: Closure Time, dtype: object
Missing values: 0


In [ ]:
df_copy['Closure Time'] = pd.to_timedelta(
    df_copy['Closure Time'].astype(str),
    errors='coerce')

In [ ]:
print(df_copy['Closure Time'].dtype)
print(df_copy['Closure Time'].head(10))
print("Missing values:", df_copy['Closure Time'].isna().sum())

timedelta64[ns]
0   0 days 12:55:00
1   0 days 22:11:00
2   0 days 12:24:00
3   0 days 15:22:00
4   0 days 17:10:00
5   0 days 10:29:00
6   0 days 09:00:00
7   0 days 12:06:00
8   0 days 09:15:00
9   0 days 08:33:00
Name: Closure Time, dtype: timedelta64[ns]
Missing values: 0


Incision Time:

In [ ]:
df_copy['Incision Time'] = pd.to_datetime(
    df_copy['Incision Time'],
    format='%H:%M:%S',
    errors='coerce'
).dt.time

In [ ]:
print(df_copy['Incision Time'].dtype)
print(df_copy['Incision Time'].head(10))
print("Missing values:", df_copy['Incision Time'].isna().sum())

object
0    11:18:00
1    20:21:00
2    11:40:00
3    14:51:00
4    16:40:00
5    09:39:00
6    07:36:00
7    11:31:00
8    07:24:00
9    07:41:00
Name: Incision Time, dtype: object
Missing values: 0


In [ ]:
df_copy['Incision Time'] = pd.to_timedelta(
    df_copy['Incision Time'].astype(str),
    errors='coerce')

In [ ]:
print(df_copy['Incision Time'].dtype)
print(df_copy['Incision Time'].head(10))
print("Missing values:", df_copy['Incision Time'].isna().sum())

timedelta64[ns]
0   0 days 11:18:00
1   0 days 20:21:00
2   0 days 11:40:00
3   0 days 14:51:00
4   0 days 16:40:00
5   0 days 09:39:00
6   0 days 07:36:00
7   0 days 11:31:00
8   0 days 07:24:00
9   0 days 07:41:00
Name: Incision Time, dtype: timedelta64[ns]
Missing values: 0


Planned Start Time for Doctor Block:

In [ ]:
df_copy['Planned Start Time for Doctor Block'] = pd.to_datetime(
    df_copy['Planned Start Time for Doctor Block'],
    format='%H:%M:%S',
    errors='coerce'
).dt.time

In [ ]:
print(df_copy['Planned Start Time for Doctor Block'].dtype)
print(df_copy['Planned Start Time for Doctor Block'].head(10))
print("Missing values:", df_copy['Planned Start Time for Doctor Block'].isna().sum())

object
0    07:00:00
1    15:00:00
2    10:40:00
3    13:00:00
4    16:10:00
5    07:00:00
6    07:00:00
7    07:00:00
8    07:00:00
9    07:00:00
Name: Planned Start Time for Doctor Block, dtype: object
Missing values: 0


In [ ]:
df_copy['Planned Start Time for Doctor Block'] = pd.to_timedelta(
    df_copy['Planned Start Time for Doctor Block'].astype(str),
    errors='coerce')

In [ ]:
print(df_copy['Planned Start Time for Doctor Block'].dtype)
print(df_copy['Planned Start Time for Doctor Block'].head(10))
print("Missing values:", df_copy['Planned Start Time for Doctor Block'].isna().sum())

timedelta64[ns]
0   0 days 07:00:00
1   0 days 15:00:00
2   0 days 10:40:00
3   0 days 13:00:00
4   0 days 16:10:00
5   0 days 07:00:00
6   0 days 07:00:00
7   0 days 07:00:00
8   0 days 07:00:00
9   0 days 07:00:00
Name: Planned Start Time for Doctor Block, dtype: timedelta64[ns]
Missing values: 0


Actual Surgery Room Entry Time:

In [ ]:
df_copy['Actual Surgery Room Entry Time'] = pd.to_datetime(
    df_copy['Actual Surgery Room Entry Time'],
    format='%H:%M:%S',
    errors='coerce'
).dt.time

In [ ]:
print(df_copy['Actual Surgery Room Entry Time'].dtype)
print(df_copy['Actual Surgery Room Entry Time'].head(10))
print("Missing values:", df_copy['Actual Surgery Room Entry Time'].isna().sum())

object
0    11:00:46
1    20:11:08
2    08:56:12
3    14:38:54
4    16:22:00
5    09:15:00
6    07:16:00
7    11:22:00
8    07:04:00
9    07:20:00
Name: Actual Surgery Room Entry Time, dtype: object
Missing values: 0


In [ ]:
df_copy['Actual Surgery Room Entry Time'] = pd.to_timedelta(
    df_copy['Actual Surgery Room Entry Time'].astype(str),
    errors='coerce')

In [ ]:
print(df_copy['Actual Surgery Room Entry Time'].dtype)
print(df_copy['Actual Surgery Room Entry Time'].head(10))
print("Missing values:", df_copy['Actual Surgery Room Entry Time'].isna().sum())

timedelta64[ns]
0   0 days 11:00:46
1   0 days 20:11:08
2   0 days 08:56:12
3   0 days 14:38:54
4   0 days 16:22:00
5   0 days 09:15:00
6   0 days 07:16:00
7   0 days 11:22:00
8   0 days 07:04:00
9   0 days 07:20:00
Name: Actual Surgery Room Entry Time, dtype: timedelta64[ns]
Missing values: 0


In [ ]:
# List of columns that contain values in hours format only
time_columns = [
    'Actual Surgery Room Entry Time',
    'Planned Start Time for Doctor Block',
    'Incision Time',
    'Closure Time',
    'Planned End Time for Doctor Block']

In [ ]:
import pandas as pd

# List of columns being checked
time_cols = ["Entry DateTime", "Incision DateTime", "Closure DateTime", "Exit DateTime"]

# Columns in datetime format
for col in time_cols:
    df_copy[col] = pd.to_datetime(df_copy[col], errors='coerce')

# Error count and list of deleted cells
internal_corrections = 0
external_corrections = 0
deleted_cells = []

# Cleaning up internal inconsistencies within a row
def clean_internal_conflicts(row):
    global internal_corrections
    for i in range(len(time_cols) - 1):
        col1 = time_cols[i]
        col2 = time_cols[i + 1]
        t1 = row[col1]
        t2 = row[col2]

        if pd.notnull(t1) and pd.notnull(t2) and t1 > t2:
            prev_time = row[time_cols[i - 1]] if i > 0 else None
            next_time = row[time_cols[i + 2]] if i + 2 < len(time_cols) else None

            if prev_time is not None and pd.notnull(prev_time) and prev_time > t1:
                bad_col = col1
            elif next_time is not None and pd.notnull(next_time) and t2 > next_time:
                bad_col = col2
            else:
                bad_col = col1  # default
            deleted_cells.append((row.name, bad_col, row[bad_col]))
            row[bad_col] = pd.NaT
            internal_corrections += 1
    return row

df_copy = df_copy.apply(clean_internal_conflicts, axis=1)

# Conflict between current Exit and next Entry
df_copy["next_Entry DateTime"] = df_copy["Entry DateTime"].shift(-1)

def fix_external_conflict(row):
    global external_corrections
    if pd.notnull(row["Exit DateTime"]) and pd.notnull(row["next_Entry DateTime"]):
        if row["Exit DateTime"] > row["next_Entry DateTime"]:
            deleted_cells.append((row.name, "Exit DateTime", row["Exit DateTime"]))
            row["Exit DateTime"] = pd.NaT
            external_corrections += 1
    return row

df_copy = df_copy.apply(fix_external_conflict, axis=1)
df_copy.drop(columns=["next_Entry DateTime"], inplace=True)

# Conflict between current Entry and previous Exit → Always delete the Exit of the previous line
df_copy["prev_Exit DateTime"] = df_copy["Exit DateTime"].shift(1)

def fix_inter_row_conflict_strict(row):
    global external_corrections
    entry = row["Entry DateTime"]
    prev_exit = row["prev_Exit DateTime"]

    if pd.notnull(entry) and pd.notnull(prev_exit) and entry < prev_exit:
        idx_prev = row.name - 1
        deleted_cells.append((idx_prev, "Exit DateTime", df_copy.at[idx_prev, "Exit DateTime"]))
        df_copy.at[idx_prev, "Exit DateTime"] = pd.NaT
        external_corrections += 1

    return row

df_copy = df_copy.apply(fix_inter_row_conflict_strict, axis=1)
df_copy.drop(columns=["prev_Exit DateTime"], inplace=True)

# Print summary:
print(f"Cleaning finished:")
print(f"{internal_corrections} Internal corrections were made within lines.")
print(f"{external_corrections} Corrections of conflicts between lines have been made.")

# Show an example of deleted entries
if deleted_cells:
    print("\n Example of deleted values (up to the first 5):")
    for row_idx, col, val in deleted_cells[:5]:
        print(f" line {row_idx}, column '{col}',Deleted entry: {val}")
else:
    print("No entries were deleted.")

Cleaning finished:
34 Internal corrections were made within lines.
1023 Corrections of conflicts between lines have been made.

 Example of deleted values (up to the first 5):
 line 99, column 'Entry DateTime',Deleted entry: 2017-10-01 07:31:01
 line 672, column 'Entry DateTime',Deleted entry: 2017-07-10 13:52:01
 line 1107, column 'Entry DateTime',Deleted entry: 2017-01-16 15:41:01
 line 2089, column 'Entry DateTime',Deleted entry: 2017-01-29 18:23:01
 line 2170, column 'Entry DateTime',Deleted entry: 2017-10-22 18:28:01


SP, only consider shift boundaries!

In [ ]:
# Checking how many missing values there are in the shift columns
missing_values = df_copy[['Planned Start Time for Doctor Block', 'Planned End Time for Doctor Block']].isna().sum()
print("Missing values at shift boundaries:\n", missing_values)

Missing values at shift boundaries:
 Planned Start Time for Doctor Block    0
Planned End Time for Doctor Block      0
dtype: int64


In [ ]:
# Convert date columns to datetime format
date_columns = ["Actual Surgery Room Entry Date", "Planned Start Date for Doctor Block"]
for col in date_columns:
    df_copy[col] = pd.to_datetime(df[col], errors='coerce')  # Save the date only

In [ ]:
print(df_copy['Planned Start Date for Doctor Block'].dtype)

datetime64[ns]


In [ ]:
import pandas as pd
import numpy as np

# Convert columns to dates:
datetime_cols = [
    "Actual Surgery Room Entry Date",
    "End of Surgery Date (Exit from OR)",
    "Planned Start Date for Doctor Block",
    "Planned End Date for Doctor Block"
]
for col in datetime_cols:
    df_copy[col] = pd.to_datetime(df_copy[col], errors='coerce')

# Creating new consolidated columns:
df_copy["Planned Start DateTime"] = df_copy["Planned Start Date for Doctor Block"] + pd.to_timedelta(df_copy["Planned Start Time for Doctor Block"].astype(str), errors='coerce')
df_copy["Planned End DateTime"] = df_copy["Planned End Date for Doctor Block"] + pd.to_timedelta(df_copy["Planned End Time for Doctor Block"].astype(str), errors='coerce')

# Actual full surgery duration without shift restrictions:
df_copy["S_p_raw"] = df_copy["Closure DateTime"] - df_copy["Incision DateTime"]

# Convert duration to decimal hours:
df_copy["S_p_raw_hours"] = df_copy["S_p_raw"].dt.total_seconds() / 3600
df_copy["S_p_raw_hours"] = df_copy["S_p_raw_hours"].round(5)

# Actual duration of surgery within the block (shift) boundaries:
df_copy["S_p_limited"] = (
    df_copy[["Closure DateTime", "Planned End DateTime"]].min(axis=1) -
    df_copy[["Incision DateTime", "Planned Start DateTime"]].max(axis=1)
).clip(lower=pd.Timedelta(0))

df_copy.loc[
    df_copy[['Incision DateTime', 'Closure DateTime', 'Planned Start DateTime', 'Planned End DateTime']].isnull().any(axis=1),
    "S_p_limited"
] = pd.NaT

df_copy["S_p_hours_limited"] = df_copy["S_p_limited"].dt.total_seconds() / 3600
df_copy.loc[df_copy["S_p_limited"].isna(), "S_p_hours_limited"] = np.nan
df_copy["S_p_hours_limited"] = df_copy["S_p_hours_limited"].round(5)

df_sp_sum = df_copy.groupby(
    ['Actual Surgery Room Entry Date', 'Actual Operating Room Number'],
    as_index=False
)['S_p_hours_limited'].sum()

df_sp_sum.rename(columns={'S_p_hours_limited': 'total_SP_per_day_room'}, inplace=True)

assert not df_sp_sum.duplicated(subset=["Actual Surgery Room Entry Date", "Actual Operating Room Number"]).any()

df_copy = df_copy.merge(
    df_sp_sum,
    on=['Actual Surgery Room Entry Date', 'Actual Operating Room Number'],
    how='left'
)

SUP:

In [ ]:
# Now convert all datetime fields
dt_cols = [
    "Planned Start DateTime", "Planned End DateTime",
    "Entry DateTime", "Exit DateTime",
    "Incision DateTime", "Closure DateTime"
]
for col in dt_cols:
    df_copy[col] = pd.to_datetime(df_copy[col], errors='coerce')

df_copy["SU_p_before_limited"] = (
    (df_copy["Incision DateTime"] - df_copy["Entry DateTime"]).clip(lower=pd.Timedelta(0)) +
    (df_copy["Exit DateTime"] - df_copy["Closure DateTime"]).clip(lower=pd.Timedelta(0))
)
df_copy["SU_p_before_hours"] = df_copy["SU_p_before_limited"].dt.total_seconds() / 3600

prep_start = df_copy[["Entry DateTime", "Planned Start DateTime"]].max(axis=1)
prep_end = df_copy[["Incision DateTime", "Planned End DateTime"]].min(axis=1)
prep_duration = (prep_end - prep_start).clip(lower=pd.Timedelta(0))

post_start = df_copy[["Closure DateTime", "Planned Start DateTime"]].max(axis=1)
post_end = df_copy[["Exit DateTime", "Planned End DateTime"]].min(axis=1)
post_duration = (post_end - post_start).clip(lower=pd.Timedelta(0))

df_copy["SU_p_after_limited"] = prep_duration + post_duration
df_copy["SU_p_after_hours"] = df_copy["SU_p_after_limited"].dt.total_seconds() / 3600

df_copy.loc[df_copy[[
    'Entry DateTime', 'Incision DateTime', 'Closure DateTime', 'Exit DateTime',
    'Planned Start DateTime', 'Planned End DateTime'
]].isnull().any(axis=1), "SU_p_after_limited"] = pd.NaT

df_copy["SU_p_after_hours"] = df_copy["SU_p_after_limited"].dt.total_seconds() / 3600
df_copy.loc[df_copy["SU_p_after_limited"].isna(), "SU_p_after_hours"] = np.nan

su_grouped = df_copy.groupby(
    ["Actual Surgery Room Entry Date", "Actual Operating Room Number"],
    as_index=False
)["SU_p_after_hours"].sum().rename(columns={"SU_p_after_hours": "SU_p_limited_hours"})

df_copy = df_copy.merge(
    su_grouped,
    on=["Actual Surgery Room Entry Date", "Actual Operating Room Number"],
    how="left"
)

print(df_copy[[
    "Actual Surgery Room Entry Date", "Actual Operating Room Number",
    "SU_p_before_hours", "SU_p_after_hours", "SU_p_limited_hours"
]].head())

  Actual Surgery Room Entry Date  Actual Operating Room Number  \
0                     2017-06-18                         20002   
1                     2017-06-28                         20009   
2                     2017-06-18                         20014   
3                     2017-06-18                         20002   
4                     2017-09-14                         20002   

   SU_p_before_hours  SU_p_after_hours  SU_p_limited_hours  
0           0.603889          0.287222            0.671389  
1                NaN               NaN            1.040278  
2           2.880000          1.100000            2.679167  
3                NaN               NaN            0.671389  
4                NaN               NaN            0.933333  


SHR:

In [ ]:
# Calculate block duration in hours
df_copy["Block Duration Hours"] = (
    df_copy["Planned End DateTime"] - df_copy["Planned Start DateTime"]
).dt.total_seconds() / 3600

# Get max block duration per surgeon per day and room
df_max = df_copy.groupby(
    ['Main Surgeon Code', 'Actual Surgery Room Entry Date', 'Actual Operating Room Number'],
    as_index=False
)['Block Duration Hours'].max()

df_max.rename(columns={'Block Duration Hours': 'max_hours'}, inplace=True)

#Sum the max values per day and room
df_sum = df_max.groupby(
    ['Actual Surgery Room Entry Date', 'Actual Operating Room Number'],
    as_index=False
)['max_hours'].sum()

# Rename the result column to SH_r_hours
df_sum.rename(columns={'max_hours': 'SH_r_hours'}, inplace=True)

# Merge SH_r_hours back to df_copy
df_copy = df_copy.merge(df_sum, on=['Actual Surgery Room Entry Date', 'Actual Operating Room Number'], how='left')

Utilization rate:

In [ ]:
print(df_copy.columns.tolist())

['patient_id', 'Site Code', 'Main Surgeon Code', 'Activity Code', 'Activity Type Code', 'Planned SU Time (Large/Medium/Small)', 'Pre-Surgery Patient File (External)', 'Pre-Surgery Admission Date', 'Height', 'Weight', 'Patient Age (on Surgery Day)', 'Background Diseases/Diagnoses', 'Planned Surgery Date', 'Planned Surgery Time', 'Surgery Admission Date', 'Administrative Admission Time', 'Planned Operating Room Number', 'Pre-Surgery Hospitalization Admission Date', 'Pre-Surgery Hospitalization Admission Time', 'Pre-Surgical Admission Time Before Surgery', 'Surgical Team Codes', 'Anesthesiologist Code', 'Anesthesia Code', 'Type of Anesthesia', 'Cancellation Reason on Surgery Day', 'Actual Operating Room Number', 'Actual Surgery Room Entry Date', 'Actual Surgery Room Entry Time', 'Incision Time', 'Closure Time', 'End of Surgery Date (Exit from OR)', 'End of Surgery Time (Exit from OR)', 'Planned Surgery Duration', 'Recovery Room Entry Date', 'Recovery Room Entry Time', 'Recovery Room Exit 

In [ ]:
# Clean the data before utilization calculation

# Remove rows where there was a No-Show
num_noshow = df_copy["No Show"].sum()
print(f"Removed {num_noshow} rows due to No-Show status.")
df_copy = df_copy[df_copy["No Show"] != True]

# Remove rows with missing time components needed for utilization calculation
required_cols = [
    "Incision DateTime",
    "Closure DateTime",
    "Planned Start DateTime",
    "Planned End DateTime",
    "Actual Surgery Room Entry Date",
    "Actual Surgery Room Entry Time",
    "End of Surgery Time (Exit from OR)"
]
df_copy = df_copy.dropna(subset=required_cols)

print(f"Remaining rows for utilization calculation: {len(df_copy)}")

if all(col in df_copy.columns for col in ["total_SP_per_day_room", "SU_p_limited_hours", "SH_r_hours"]):
    df_copy["daily_utilization_rate"] = (
        (df_copy["total_SP_per_day_room"] + df_copy["SU_p_limited_hours"]) /
        df_copy["SH_r_hours"]
    ) * 100
    print(f"Calculated utilization for {df_copy['daily_utilization_rate'].notna().sum()} rows.")

    df_copy.loc[df_copy["SH_r_hours"].isna(), "daily_utilization_rate"] = np.nan
    df_copy.loc[df_copy["SH_r_hours"] == 0, "daily_utilization_rate"] = np.nan
else:
    print("Missing columns required to calculate utilization")

Removed 0 rows due to No-Show status.
Remaining rows for utilization calculation: 13587
Calculated utilization for 13587 rows.


In [ ]:
df_copy = df_copy.merge(df_copy[["Planned Start Date for Doctor Block",
                                        "Actual Operating Room Number",
                                        "daily_utilization_rate"]],
                        on=["Planned Start Date for Doctor Block", "Actual Operating Room Number"],
                        how="left")

In [ ]:
from datetime import timedelta

for col in time_columns:
    if col in df_copy.columns:
        df_copy[col] = df_copy[col].apply(
            lambda x: f"{int(x.total_seconds() // 3600):02}:{int((x.total_seconds() % 3600) // 60):02}:{int(x.total_seconds() % 60):02}"
            if isinstance(x, timedelta) else (x if isinstance(x, str) else "")
        )

In [ ]:
df_copy.to_excel("2017_data_with_UTR.xlsx", index=False)